In [1]:
import pandas as pd

In [2]:
!pwd

/home/panayot/Documents/site_similarity/notebooks/node_features_graphs/corpus 2018


In [3]:
! cd ../../

In [4]:
!pwd

/home/panayot/Documents/site_similarity/notebooks/node_features_graphs/corpus 2018


In [5]:
node_features_file = "../../generate_node_features/corpus_2018_audience_overlap_level_0_and_1_node_features.csv"
edge_file = "../../generate_node_features/corpus_2018_audience_overlap_level_0_and_1_edges.csv"

In [6]:
node_features_df = pd.read_csv(node_features_file, index_col=0)

In [7]:
node_features_df.head()

,alexa_ranks,daily_pageviews_per_visitors,daily_time_on_sites,total_sites_linking_ins,bounce_rate
urls,,,,,
crotonmusiccenter.com,NaN,NaN,NaN,NaN,NaN
gracefuneralhome.com,NaN,NaN,NaN,4.0,NaN
haaretz.com,11522.0,1.5,152.0,13974.0,0.650
osubeavers.com,422967.0,2.5,299.0,430.0,0.471
prophezine.com,8782212.0,1.0,NaN,209.0,NaN


In [8]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11865 entries, crotonmusiccenter.com to yorkregion.com
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alexa_ranks                   8658 non-null   float64
 1   daily_pageviews_per_visitors  8659 non-null   float64
 2   daily_time_on_sites           6459 non-null   float64
 3   total_sites_linking_ins       11470 non-null  float64
 4   bounce_rate                   6023 non-null   float64
dtypes: float64(5)
memory usage: 556.2+ KB


In [9]:
node_features_df.alexa_ranks = node_features_df.alexa_ranks.fillna(0)
node_features_df.total_sites_linking_ins = node_features_df.total_sites_linking_ins.fillna(0)

In [10]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11865 entries, crotonmusiccenter.com to yorkregion.com
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alexa_ranks                   11865 non-null  float64
 1   daily_pageviews_per_visitors  8659 non-null   float64
 2   daily_time_on_sites           6459 non-null   float64
 3   total_sites_linking_ins       11865 non-null  float64
 4   bounce_rate                   6023 non-null   float64
dtypes: float64(5)
memory usage: 556.2+ KB


# Normalizing features

In [11]:
node_features_df['normalized_alexa_rank'] = node_features_df['alexa_ranks'].apply(lambda x: 1/x if x else 0)

In [12]:
import math

node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_ins'].apply(lambda x: math.log2(x) if x else 0)

---

In [13]:
edge_df = pd.read_csv(edge_file)

edge_df.head()

,source,target
0,villagevoice.com,nylon.com
1,villagevoice.com,slantmagazine.com
2,villagevoice.com,gothamist.com
3,villagevoice.com,screendaily.com
4,villagevoice.com,amny.com


In [14]:
edge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20399 entries, 0 to 20398
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  20399 non-null  object
 1   target  20399 non-null  object
dtypes: object(2)
memory usage: 318.9+ KB


In [15]:
import stellargraph as sg

In [16]:
G = sg.StellarGraph(node_features_df[['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edge_df)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 11865, Edges: 20399

 Node types:
  default: [11865]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [20399]
        Weights: all 1 (default)
        Features: none


# Unsupervised Attrib2Vec

In [17]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

1. Specify the other optional parameter values:
* root nodes
* the number of walks to take per node
* the length of each walk
* random seed

In [21]:
nodes = list(G.nodes())
number_of_walks = 10
length = 100

2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.

In [22]:
unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length=length, number_of_walks=number_of_walks
)

3. Create a node pair generator:

In [23]:
batch_size = 50
# epochs = 4
num_samples = [10, 5]

In [25]:
generator = Attri2VecLinkGenerator(G, batch_size)
train_gen = generator.flow(unsupervised_samples)

In [26]:
layer_sizes = [128]
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)

In [27]:
# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

In [28]:
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [29]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

In [ ]:
history = model.fit(
    train_gen,
    epochs=8,
    verbose=2,
    use_multiprocessing=False,
    workers=1,
    shuffle=True,
)

"""
1. (run) previous before normalization

WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 1899 steps
Epoch 1/8
1899/1899 - 47s - loss: 0.7380 - binary_accuracy: 0.5427
Epoch 2/8
1899/1899 - 46s - loss: 0.6368 - binary_accuracy: 0.6424
Epoch 3/8
1899/1899 - 47s - loss: 0.5929 - binary_accuracy: 0.6680
Epoch 4/8
1899/1899 - 48s - loss: 0.5694 - binary_accuracy: 0.6800
Epoch 5/8
1899/1899 - 52s - loss: 0.5564 - binary_accuracy: 0.6865
Epoch 6/8
1899/1899 - 47s - loss: 0.5442 - binary_accuracy: 0.6933
Epoch 7/8
1899/1899 - 48s - loss: 0.5399 - binary_accuracy: 0.6941
Epoch 8/8
1899/1899 - 47s - loss: 0.5272 - binary_accuracy: 0.7013

2 (run) after normalization but without parameters tuning

WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 1899 steps
Epoch 1/8
1899/1899 - 49s - loss: 0.6926 - binary_accuracy: 0.5304
Epoch 2/8
1899/1899 - 52s - loss: 0.6343 - binary_accuracy: 0.5872
Epoch 3/8
1899/1899 - 43s - loss: 0.6147 - binary_accuracy: 0.6016
Epoch 4/8
1899/1899 - 43s - loss: 0.5955 - binary_accuracy: 0.6457
Epoch 5/8
1899/1899 - 43s - loss: 0.5755 - binary_accuracy: 0.6763
Epoch 6/8
1899/1899 - 44s - loss: 0.5604 - binary_accuracy: 0.6914
Epoch 7/8
1899/1899 - 44s - loss: 0.5486 - binary_accuracy: 0.7005
Epoch 8/8
1899/1899 - 43s - loss: 0.5390 - binary_accuracy: 0.7082

"""

Epoch 1/8
469854/469854 - 4185s - loss: 0.7051 - binary_accuracy: 0.5489
Epoch 2/8


In [ ]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

In [ ]:
node_gen = Attri2VecNodeGenerator(G, batch_size).flow(node_features_df.index)
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=1)

In [ ]:
node_embeddings[213]

In [ ]:
embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings))

In [ ]:
embeddings_wv['villagevoice.com']

In [ ]:
class ModelWrapper:
    def __init__(self, embeddings_wv):
        self.wv = embeddings_wv
        
    def __str__(self):
        return 'Unsupervised Attrib2Vec'

In [31]:
import sys,os
sys.path.append("C:\\Users\\Paco\\Documents\\site_similarity")

In [32]:
from utils.notebook_utils import train_model

In [ ]:
data_year = '2018'
node2vec_model = ModelWrapper(embeddings_wv)

In [33]:
! cd

C:\Users\Paco\Documents\site_similarity\notebooks\node_features_graphs


In [34]:
%run ../../utils/notebook_utils.py

In [ ]:
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
result_report = []

clf = LogisticRegressionCV(Cs=10, cv=5, scoring="accuracy", multi_class="ovr", max_iter=300, random_state=42)
result_report.append([
    str(node2vec_model),
    'LogisticRegression CV = 5',
    *list(train_model(clf, node2vec_model=node2vec_model, data_year=data_year).values())
]);

clf2 = LogisticRegressionCV(Cs=10, cv=10, scoring="accuracy", multi_class="ovr", max_iter=300, random_state=42)
result_report.append([
    str(node2vec_model),
    'LogisticRegression CV = 10',
    *list(train_model(clf2, node2vec_model=node2vec_model, data_year=data_year).values())
]);

tree_clf = GradientBoostingClassifier(random_state=42)
result_report.append([
    str(node2vec_model),
    'GradientBoostingClassifier',
    *list(train_model(tree_clf, node2vec_model=node2vec_model, data_year=data_year).values())
]);

svm_clf = svm.SVC(decision_function_shape='ovo', probability=True, random_state=42)
result_report.append([
    str(node2vec_model),
    'SVC ovo',
    *list(train_model(svm_clf, node2vec_model=node2vec_model, data_year=data_year).values())
]);

model_res = pd.DataFrame(result_report,
                    columns=["Feature", "Classifier", "Accuracy", "Balanced Accuracy score",
                             "F1 micro score", "F1 macro score", "F1 weighted score", "MAE", "Confusion matrix"])

In [ ]:
model_res.head()